In [2]:
import pandas as pd
import numpy as np
import networkx as nx
from node2vec import Node2Vec
import os
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
from pycaret import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [4]:
# Loading the phase2 data

loc_phase2="Phase2_Provenance.csv"

phase2_raw= pd.read_csv(loc_phase2)


C:\Users\Acer\AppData\Local\Temp\ipykernel_38656\1848470031.py:5: DtypeWarning: Columns (24,27,34) have mixed types. Specify dtype option on import or set low_memory=False.
  phase2_raw= pd.read_csv(loc_phase2)


In [5]:
phase2_raw

,id,type,from,to,uid,egid,exe,gid,euid,name,pid,seen time,source,ppid,command line,start time,event id,time,operation,path,subtype,permissions,epoch,version,flags,local address,remote port,protocol,remote address,local port,tgid,fd,mode,label,subLabel
0,ca5b322fa4d4cb63aa5dd9fbd88e37e4,Process,NaN,NaN,0.0,0.0,/usr/bin/dash,0.0,0.0,ethtool,917.0,1.701469e+09,syscall,913.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1,e36715b62c8cdc32e47483b3600712f1,Process,NaN,NaN,0.0,0.0,/usr/bin/dash,0.0,0.0,ethtool,917.0,NaN,syscall,913.0,/bin/sh /etc/network/if-up.d/ethtool,1.701469e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,NaN,WasTriggeredBy,e36715b62c8cdc32e47483b3600712f1,ca5b322fa4d4cb63aa5dd9fbd88e37e4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,syscall,NaN,NaN,NaN,246.0,1.701469e+09,execve,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3,216f4d4893942a66d649c4e02b5722c4,Artifact,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,syscall,NaN,NaN,NaN,NaN,NaN,NaN,/etc/network/if-up.d/ethtool,file,755.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,NaN,Used,e36715b62c8cdc32e47483b3600712f1,216f4d4893942a66d649c4e02b5722c4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,syscall,NaN,NaN,NaN,246.0,1.701469e+09,load,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196730,e97a4f8ea72d1ccbbd9a4fd26692c38e,Process,NaN,NaN,0.0,0.0,/usr/bin/systemd-tty-ask-password-agent,0.0,0.0,systemd-tty-ask,244020.0,NaN,syscall,244019.0,/bin/systemd-tty-ask-password-agent --watch,1.701728e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
196731,NaN,WasTriggeredBy,e97a4f8ea72d1ccbbd9a4fd26692c38e,cc3aa5736c336ef1edaa2d2c54191f8e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,syscall,NaN,NaN,NaN,404003.0,1.701728e+09,execve,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
196732,12b6df6ce5e36fc5608f5c2d916718ae,Artifact,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,syscall,NaN,NaN,NaN,NaN,NaN,NaN,/bin/systemd-tty-ask-password-agent,file,755.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
196733,NaN,Used,e97a4f8ea72d1ccbbd9a4fd26692c38e,12b6df6ce5e36fc5608f5c2d916718ae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,syscall,NaN,NaN,NaN,404003.0,1.701728e+09,load,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [6]:
# Separating the nodes and edges

raw_edges= phase2_raw.dropna(subset=["to"])
node_mask = phase2_raw.index.isin(raw_edges.index)
raw_nodes=phase2_raw[~node_mask]

In [7]:
# Processing edges; Dropping some null features and encoding valuable edge features
edges=raw_edges.copy()
nodes=raw_nodes.copy()

edge_attributes_to_drop=["id","uid","egid","exe","gid","euid","name","pid","seen time", "source","ppid",
                         "command line","start time","event id","path","subtype","permissions","epoch","version","flags",
                        "local address","remote port","protocol","remote address","local port","tgid","fd","mode"]

edge_attributes_to_encode=["type","operation"]

edges.drop(columns=edge_attributes_to_drop,inplace=True)

type_encoder = LabelEncoder()
operation_encoder = LabelEncoder()

edges["type"]=type_encoder.fit_transform(edges["type"])
edges["operation"]=operation_encoder.fit_transform(edges["operation"])

In [8]:
#Dropping some null features from nodes
node_attributes_to_drop=["from", "to" , "pid" , "seen time" , "source", "ppid", "command line", "start time" ,
                        "event id" , "time" , "operation" , "version", "flags","local address" , "local port",
                        "tgid","fd","mode","epoch","protocol"]
process_attributes=["uid", "egid" , "exe" , "gid" , "euid" , "name" ,]
artifact_attributes=["path","permissions","epoch","remote port","protocol","remote address"]

nodes.drop(columns=node_attributes_to_drop,inplace=True)

In [9]:
# Encoding the node features

attr_to_encode=["type","subtype","uid","egid","exe","gid","euid","name","path","subtype","permissions","remote port","remote address"]

nodes_encoder = LabelEncoder()
for i in attr_to_encode:
    nodes[i]=nodes_encoder.fit_transform(nodes[i])



In [10]:
# Creating the graph using networkX

graph= nx.from_pandas_edgelist(edges,source="from",target="to", edge_attr=["type","operation"], create_using=nx.MultiDiGraph())

In [11]:
print("The number of graph nodes are: {} and the number of edges are: {}".format(graph.number_of_nodes(), graph.number_of_edges()))

The number of graph nodes are: 53286 and the number of edges are: 143449


In [12]:
# Assigning the node features to the graph's node attribute

node_attributes_dict = nodes.drop(columns=["label","subLabel"]).set_index('id').to_dict(orient='index')
nx.set_node_attributes(graph,node_attributes_dict)

In [13]:
for node, attrs in graph.nodes(data=True):
    print(f"Node {node}: {attrs}")

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [14]:
# Using node2vec on the graph to get node embeddings 
node2vec_phase2 = Node2Vec(graph, dimensions=64, walk_length=10, num_walks=100, workers=8)

Computing transition probabilities: 100%|███████████████████████████████████████| 53286/53286 [02:02<00:00, 433.90it/s]


In [15]:
model = node2vec_phase2.fit(window=10, min_count=1, batch_words=4)

TypeError: Word2Vec.__init__() got an unexpected keyword argument 'size'

In [ ]:
# Saving the trained Node2Vec models for further uses

model.save("./Sample_Node2Vec_64sized_phase2.model")

In [ ]:
# Loading the saved Node2Vec models 

path_to_phase2_model="./Sample_Node2Vec_64sized_phase2.model"
model2=Word2Vec.load(path_to_phase2_model)

In [ ]:
# Creating the phase2 dataframe; including the embeddings, labels, and sub-labels 

embeddings2 = {node: model2.wv[node] for node in graph.nodes()}
embedding_df2 = pd.DataFrame.from_dict(embeddings2, orient='index')

labels_dict2 = {node["id"]: node["label"] for _ ,node in nodes.iterrows()}
sub_labels_dict2 = {node["id"]: node["subLabel"] for _ ,node in nodes.iterrows()}

label_df2 = pd.DataFrame.from_dict(labels_dict2, orient='index', columns=['label'])
sub_label_df2 = pd.DataFrame.from_dict(sub_labels_dict2, orient='index', columns=['subLabel'])

phase2_processed = embedding_df2.join(label_df2).join(sub_label_df2)

In [ ]:
phase2_processed

In [ ]:
# The first model; data consists of all attack sub-classes and only from phase2
# Binary classification; all stages of attack are labelled as 1 

print("Training the first model")
s = setup(data = phase2_processed.drop(columns=["subLabel"]), target = 'label', session_id=123)
best_model = compare_models()